In [195]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, LSTM

In [196]:
#tf.config.experimental.set_visible_devices([], 'GPU')

In [197]:
data = pd.read_json('archive/Sarcasm_Headlines_Dataset.json', lines = True)

In [198]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26709 entries, 0 to 26708
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   article_link  26709 non-null  object
 1   headline      26709 non-null  object
 2   is_sarcastic  26709 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 626.1+ KB


In [199]:
x = data['headline']
y = data['is_sarcastic']

In [200]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 1)

x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

training_sequences = tokenizer.texts_to_sequences(x_train)
training_padded = pad_sequences(training_sequences, 100)

testing_sequences = tokenizer.texts_to_sequences(x_test)
testing_padded = pad_sequences(testing_sequences, 100)

model = tf.keras.models.Sequential()
model.add(Embedding(100000, 12, input_length=100))
model.add(LSTM(128))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
history_data = model.fit(training_padded, y_train, 
                         batch_size = 512, epochs = 25, 
                         validation_data = (testing_padded, y_test))

Epoch 1/25
47/47 [==============================] - 2s 29ms/step - loss: 0.6861 - accuracy: 0.5504 - val_loss: 0.6930 - val_accuracy: 0.5792
Epoch 2/25
47/47 [==============================] - 1s 23ms/step - loss: 0.6119 - accuracy: 0.7861 - val_loss: 0.4209 - val_accuracy: 0.8180
Epoch 3/25
47/47 [==============================] - 1s 23ms/step - loss: 0.4172 - accuracy: 0.8256 - val_loss: 0.3462 - val_accuracy: 0.8499
Epoch 4/25
47/47 [==============================] - 1s 23ms/step - loss: 0.2152 - accuracy: 0.9259 - val_loss: 0.3344 - val_accuracy: 0.8615
Epoch 5/25
47/47 [==============================] - 1s 24ms/step - loss: 0.1422 - accuracy: 0.9545 - val_loss: 0.3523 - val_accuracy: 0.8596
Epoch 6/25
47/47 [==============================] - 1s 23ms/step - loss: 0.0942 - accuracy: 0.9716 - val_loss: 0.4111 - val_accuracy: 0.8589
Epoch 7/25
47/47 [==============================] - 1s 23ms/step - loss: 0.0553 - accuracy: 0.9856 - val_loss: 0.4511 - val_accuracy: 0.8611
Epoch 8/25
47

In [ ]:
import matplotlib.pyplot as plt
def plot_data(history, string):
  plt.plot(history_data.history[string])
  plt.plot(history_data.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_data(history_data, "accuracy")
plot_data(history_data, "loss")

In [ ]:
results = model.evaluate(testing_padded, y_test)
results